In [2]:
'''
This script mainly does:
 -- The first half of this file converts the smiles to isomeric form, and save the pickles to the original location
    It is no longer needed as in our new dataset, they are already non-isomeric
 -- The second half of this file is to check no 2d-1d overlap in the dataset
'''


import torch, pickle, yaml
all_smiles_train_2d = pickle.load(open('/workspace/SMILES_dataset/train/SMILES/index.pkl', 'rb'))
all_smiles_val_2d = pickle.load(open('/workspace/SMILES_dataset/val/SMILES/index.pkl', 'rb'))
all_smiles_test_2d = pickle.load(open('/workspace/SMILES_dataset/test/SMILES/index.pkl', 'rb'))

all_chemical_names_train_2d = pickle.load(open('/workspace/SMILES_dataset/train/Chemical/index.pkl', 'rb'))
all_chemical_names_val_2d = pickle.load(open('/workspace/SMILES_dataset/val/Chemical/index.pkl', 'rb'))
all_chemical_names_test_2d = pickle.load(open('/workspace/SMILES_dataset/test/Chemical/index.pkl', 'rb'))

all_smiles_train_1d = pickle.load(open('/workspace/OneD_Only_Dataset/train/SMILES/index.pkl', 'rb'))
all_smiles_val_1d = pickle.load(open('/workspace/OneD_Only_Dataset/val/SMILES/index.pkl', 'rb'))
all_smiles_test_1d = pickle.load(open('/workspace/OneD_Only_Dataset/test/SMILES/index.pkl', 'rb'))

all_chemical_names_train_1d = pickle.load(open('/workspace/OneD_Only_Dataset/train/Chemical/index.pkl', 'rb'))
all_chemical_names_val_1d = pickle.load(open('/workspace/OneD_Only_Dataset/val/Chemical/index.pkl', 'rb'))
all_chemical_names_test_1d = pickle.load(open('/workspace/OneD_Only_Dataset/test/Chemical/index.pkl', 'rb'))

import imp
smart = imp.load_source('/root/gurusmart/MorganFP_prediction/reproduce_previous_works/smart4.5/datasets/', '/root/gurusmart/MorganFP_prediction/reproduce_previous_works/smart4.5/datasets/dataset_utils.py')
    


/tmp/ipykernel_2597/3108716532.py:27: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [6]:
all_smiles_val_1d

{12: 'O=C(O)[C@H]1O[C@H](OP(=O)(O)OP(=O)(O)OC[C@H]2O[C@@H](n3ccc(=O)[nH]c3=O)[C@H](O)[C@@H]2O)[C@H](O)[C@@H](O)[C@@H]1O',
 27: 'CC(=O)OC[C@H](N)C(=O)O',
 37: 'CC(C)[C@@H]1CC[C@@H](C)C[C@H]1O',
 43: 'OC1OC[C@@H](O)[C@H]1O',
 48: 'C=C(C)[C@@H]1CC=C(C)CC1',
 55: 'Nc1nc(=O)c2c([nH]1)NC[C@H](CNc1ccc(C(=O)NC(CCC(=O)O)C(=O)O)cc1)N2',
 59: 'Nc1ncnc2c1ncn2[C@H]1C[C@H](O)[C@@H](CO)O1',
 66: 'C[C@H](CCC(=O)O)[C@H]1CC[C@H]2[C@@H]3[C@@H](O)C[C@@H]4C[C@H](O)CC[C@]4(C)[C@H]3CC[C@@]21C',
 71: 'CC/C=C\\C/C=C\\C/C=C\\CCCCCCCC(=O)O',
 90: 'O=C1CCCCN1',
 92: 'C[C@H](CCC(=O)O)[C@H]1CC[C@H]2[C@H]3[C@H](CC[C@@]21C)[C@@]1(C)CC[C@@H](O)C[C@H]1[C@H](O)[C@@H]3O',
 102: 'COc1ccc(/C=C/C(=O)O)cc1',
 123: 'CC1(C)C2CC[C@]3(C)C(C(=O)C=C4C5C[C@@](C)(C(=O)O)CC[C@]5(C)CC[C@]43C)[C@@]2(C)CC[C@@H]1O',
 127: 'CC1=C(/C=C/C(C)=C/C=C/C(C)=C/C=C/C=C(C)/C=C/C=C(C)/C=C/C2=C(C)CCCC2(C)C)C(C)(C)CCC1',
 137: 'C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1CCC2=O',
 144: 'O=C(/C=C/c1ccccc1)c1ccccc1',
 145: 'OC[C@H]1O[C@H](OC[C@H]2O[C@@H](

In [8]:
import sys, os 
from rdkit import Chem

def isomeric_to_canonical_smiles(isomeric_smiles):

    mol = Chem.MolFromSmiles(isomeric_smiles)
    Chem.RemoveStereochemistry( mol ) 

    canonical_smiles = Chem.MolToSmiles(mol, canonical=True)
    
    return canonical_smiles

In [9]:
import tqdm

from collections import defaultdict
# build smiles and chemical names 
non_repeating_smiles = set()
for datasrc in ['2d',"1d" ]:
    for split in ['train', 'val', 'test']:
        all_smiles = locals()['all_smiles_' + split + '_' + datasrc]
        
        dataset_dir =  '/workspace/OneD_Only_Dataset/' if datasrc == '1d' else '/workspace/SMILES_dataset/'
        out_path = dataset_dir + split + '/SMILES/index.pkl'
        print(out_path)
        os.system(f"mkdir {dataset_dir + split + '/Isomeric_SMILES/'}")
        os.system(f"cp {out_path} {out_path.replace('SMILES', 'Isomeric_SMILES')}")
        
        for iter,(k,v) in enumerate(all_smiles.items()):
            canonical_smiles = isomeric_to_canonical_smiles(v)
            if canonical_smiles not in non_repeating_smiles: 
                all_smiles[k] = isomeric_to_canonical_smiles(v)
            else: # delete the NMR file if path exists
                f_path = dataset_dir + split + '/oneD_NMR/' + k + '.pt'
                if os.path.exists(f_path):
                    os.system(f"rm {f_path}")
                f_path.replace('oneD_NMR', 'HSQC')
                if os.path.exists(f_path):
                    os.system(f"rm {f_path}")
                del all_smiles[k]    
        pickle.dump(all_smiles, open(out_path, 'wb'))
        
        
# for split in ['train', 'val', 'test']:
#         valid_indices = []
#         all_smiles = locals()['all_smiles_' + split + '_1d' ]
#         dataset_dir =  '/workspace/OneD_Only_Dataset/'
#         out_path = dataset_dir + split + '/SMILES/index.pkl'
#         print(out_path)
#         os.system(f"mkdir {dataset_dir + split + '/Isomeric_SMILES/'}")
#         os.system(f"cp {out_path} {out_path.replace('SMILES', 'Isomeric_SMILES')}")
#         for k,v in tqdm.tqdm(all_smiles.items()):
#             all_smiles[k] = isomeric_to_canonical_smiles(v)
#         pickle.dump(all_smiles, open(out_path, 'wb'))

/workspace/SMILES_dataset/train/SMILES/index.pkl


mkdir: cannot create directory ‘/workspace/SMILES_dataset/train/Isomeric_SMILES/’: File exists
cp: cannot create regular file '/workspace/Isomeric_SMILES_dataset/train/Isomeric_SMILES/index.pkl': No such file or directory
[15:58:48] WARNING: not removing hydrogen atom without neighbors
[15:58:48] WARNING: not removing hydrogen atom without neighbors
[15:58:51] WARNING: not removing hydrogen atom without neighbors
[15:58:51] WARNING: not removing hydrogen atom without neighbors
[15:58:51] WARNING: not removing hydrogen atom without neighbors
[15:58:51] WARNING: not removing hydrogen atom without neighbors
[15:59:01] WARNING: not removing hydrogen atom without neighbors
[15:59:01] WARNING: not removing hydrogen atom without neighbors
[15:59:01] WARNING: not removing hydrogen atom without neighbors
[15:59:01] WARNING: not removing hydrogen atom without neighbors
[15:59:13] WARNING: not removing hydrogen atom without neighbors
[15:59:13] WARNING: not removing hydrogen atom without neighbor

/workspace/SMILES_dataset/val/SMILES/index.pkl


mkdir: cannot create directory ‘/workspace/SMILES_dataset/val/Isomeric_SMILES/’: File exists
cp: cannot create regular file '/workspace/Isomeric_SMILES_dataset/val/Isomeric_SMILES/index.pkl': No such file or directory
[15:59:33] WARNING: not removing hydrogen atom without neighbors
[15:59:33] WARNING: not removing hydrogen atom without neighbors
[15:59:33] WARNING: not removing hydrogen atom without neighbors
[15:59:33] WARNING: not removing hydrogen atom without neighbors
[15:59:34] WARNING: not removing hydrogen atom without neighbors
[15:59:34] WARNING: not removing hydrogen atom without neighbors


/workspace/SMILES_dataset/test/SMILES/index.pkl


mkdir: cannot create directory ‘/workspace/SMILES_dataset/test/Isomeric_SMILES/’: File exists
cp: cannot create regular file '/workspace/Isomeric_SMILES_dataset/test/Isomeric_SMILES/index.pkl': No such file or directory
[15:59:39] WARNING: not removing hydrogen atom without neighbors
[15:59:39] WARNING: not removing hydrogen atom without neighbors
[15:59:42] WARNING: not removing hydrogen atom without neighbors
[15:59:42] WARNING: not removing hydrogen atom without neighbors


/workspace/OneD_Only_Dataset/train/SMILES/index.pkl


[16:00:09] WARNING: not removing hydrogen atom without neighbors
[16:00:09] WARNING: not removing hydrogen atom without neighbors


/workspace/OneD_Only_Dataset/val/SMILES/index.pkl
/workspace/OneD_Only_Dataset/test/SMILES/index.pkl


In [10]:
# mol = Chem.MolFromSmiles("NC(=O)c1ccc[n+](C2OC(COP(=O)(O)OP(=O)(O)OCC3OC(n4cnc5c(N)ncnc54)C(O)C3O)C(O)C2O)c1")
# img = Draw.MolToImage(mol)
# img.show()

In [11]:
import torch, pickle, yaml, os
all_smiles_train_2d = pickle.load(open('/workspace/SMILES_dataset/train/SMILES/index.pkl', 'rb'))
all_smiles_val_2d = pickle.load(open('/workspace/SMILES_dataset/val/SMILES/index.pkl', 'rb'))
all_smiles_test_2d = pickle.load(open('/workspace/SMILES_dataset/test/SMILES/index.pkl', 'rb'))

all_chemical_names_train_2d = pickle.load(open('/workspace/SMILES_dataset/train/Chemical/index.pkl', 'rb'))
all_chemical_names_val_2d = pickle.load(open('/workspace/SMILES_dataset/val/Chemical/index.pkl', 'rb'))
all_chemical_names_test_2d = pickle.load(open('/workspace/SMILES_dataset/test/Chemical/index.pkl', 'rb'))


all_smiles_train_1d = pickle.load(open('/workspace/OneD_Only_Dataset/train/SMILES/index.pkl', 'rb'))
all_smiles_val_1d = pickle.load(open('/workspace/OneD_Only_Dataset/val/SMILES/index.pkl', 'rb'))
all_smiles_test_1d = pickle.load(open('/workspace/OneD_Only_Dataset/test/SMILES/index.pkl', 'rb'))

all_chemical_names_train_1d = pickle.load(open('/workspace/OneD_Only_Dataset/train/Chemical/index.pkl', 'rb'))
all_chemical_names_val_1d = pickle.load(open('/workspace/OneD_Only_Dataset/val/Chemical/index.pkl', 'rb'))
all_chemical_names_test_1d = pickle.load(open('/workspace/OneD_Only_Dataset/test/Chemical/index.pkl', 'rb'))
# all_smiles_val_1d

In [12]:
# merge_all_2d_smiles
all_smiles_1d=[]
for split in ['train', 'val', 'test']:
        valid_indices = []
        all_smiles = locals()['all_smiles_' + split + '_1d' ]
        dataset_dir =  f'/workspace/OneD_Only_Dataset/{split}/oneD_NMR/'
        for file_id in os.listdir(dataset_dir):
            int_id = int(file_id.split('.')[0])
            all_smiles_1d.append(all_smiles[int_id])

In [13]:
len(all_smiles_1d), len(set(all_smiles_1d))

(125413, 109816)

In [14]:
all_smiles_2d=[]
for split in ['train', 'val', 'test']:
        valid_indices = []
        all_smiles = locals()['all_smiles_' + split + '_2d' ]
        dataset_dir =  f'/workspace/SMILES_dataset/{split}/HSQC/'
        for file_id in os.listdir(dataset_dir):
            int_id = int(file_id.split('.')[0])
            all_smiles_2d.append(isomeric_to_canonical_smiles(all_smiles[int_id]))

[16:02:52] WARNING: not removing hydrogen atom without neighbors
[16:02:53] WARNING: not removing hydrogen atom without neighbors
[16:02:57] WARNING: not removing hydrogen atom without neighbors
[16:03:00] WARNING: not removing hydrogen atom without neighbors
[16:03:04] WARNING: not removing hydrogen atom without neighbors
[16:03:04] WARNING: not removing hydrogen atom without neighbors
[16:03:05] WARNING: not removing hydrogen atom without neighbors
[16:03:07] WARNING: not removing hydrogen atom without neighbors
[16:03:08] WARNING: not removing hydrogen atom without neighbors
[16:03:09] WARNING: not removing hydrogen atom without neighbors
[16:03:15] WARNING: not removing hydrogen atom without neighbors
[16:03:15] WARNING: not removing hydrogen atom without neighbors
[16:03:16] WARNING: not removing hydrogen atom without neighbors
[16:03:17] WARNING: not removing hydrogen atom without neighbors
[16:03:20] WARNING: not removing hydrogen atom without neighbors


In [15]:
len(all_smiles_2d), len(set(all_smiles_2d))

(137062, 111355)

In [16]:
all_smiles_2d=[]
for split in ['train', 'val', 'test']:
        valid_indices = []
        all_smiles = locals()['all_smiles_' + split + '_2d' ]
        dataset_dir =  f'/workspace/SMILES_dataset/{split}/HSQC/'
        for file_id in os.listdir(dataset_dir):
            int_id = int(file_id.split('.')[0])
            all_smiles_2d.append((all_smiles[int_id]))

In [17]:
len(all_smiles_2d), len(set(all_smiles_2d))

(137062, 111355)

In [18]:
overlap_2d1d = set(all_smiles_2d).intersection(set(all_smiles_1d))
len(overlap_2d1d)

29239